In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from openpharmacophore import pharmacophoric_elements as phe
from openpharmacophore.ligand_based import LigandBasedPharmacophore
from openpharmacophore.io.pharmagist import read_pharmagist

import os
import re
import pyunitwizard as puw
import numpy as np

## Reading a pharmacophore from a pharmagist mol2 file

In [3]:
files = ["elastase.mol2", "streptadivin.mol2"]
f_directory = "../Data/pharmagist/Pharmacophore"

# Load streptadivin pharmacophores
pharmacophores = read_pharmagist(os.path.join(f_directory, files[1]))
pharmacophores

[LigandBasedPharmacophore(n_elements: 9),
 LigandBasedPharmacophore(n_elements: 10),
 LigandBasedPharmacophore(n_elements: 16),
 LigandBasedPharmacophore(n_elements: 11),
 LigandBasedPharmacophore(n_elements: 13),
 LigandBasedPharmacophore(n_elements: 13)]

In [4]:
# Load elastase pharmacophores
elastase_ph = read_pharmagist(os.path.join(f_directory, files[0]))
elastase_ph

[LigandBasedPharmacophore(n_elements: 4),
 LigandBasedPharmacophore(n_elements: 21),
 LigandBasedPharmacophore(n_elements: 16),
 LigandBasedPharmacophore(n_elements: 18),
 LigandBasedPharmacophore(n_elements: 10),
 LigandBasedPharmacophore(n_elements: 12),
 LigandBasedPharmacophore(n_elements: 11),
 LigandBasedPharmacophore(n_elements: 14)]

In [5]:
len(elastase_ph)

8

In [6]:
# Load Single pharmacophore for elastase
elastase_single_ph = read_pharmagist(os.path.join(f_directory, files[0]), 0)

In [7]:
len(elastase_single_ph)

4

In [8]:
isinstance(elastase_single_ph[0], phe.HBAcceptorSphere)

True

In [9]:
# Create pharmacophore for elastase from class method
lb_pharmacophore = LigandBasedPharmacophore().from_file(os.path.join(f_directory, files[0]), index=3)
lb_pharmacophore

LigandBasedPharmacophore(n_elements: 18)

## Saving pharmacophores to pharmagist mol2 file

In [10]:
#Streptadivin pharmacophores
pharmacophores

[LigandBasedPharmacophore(n_elements: 9),
 LigandBasedPharmacophore(n_elements: 10),
 LigandBasedPharmacophore(n_elements: 16),
 LigandBasedPharmacophore(n_elements: 11),
 LigandBasedPharmacophore(n_elements: 13),
 LigandBasedPharmacophore(n_elements: 13)]

In [16]:
f_path = "../Data/pharmagist/Pharmacophore/saves/pharmacophore.mol2"

pharmagist_element_name = { # dictionary to map openphamracohpore feature names to pharmagist 
        "aromatic ring": "AR",
        "hydrophobicity": "HYD",
        "hb acceptor": "ACC",
        "hb donor": "DON",
        "positive charge": "CAT",
        "negative charge": "ANI",
        }

pharmagist_element_specs = { 
        "aromatic ring": "AR",
        "hydrophobicity": "HYD",
        "hb acceptor": "HB",
        "hb donor": "HB",
        "positive charge": "CHG",
        "negative charge": "CHG",
}

doc = [] # list to store all pharmacophores
for pharmacophore in pharmacophores:
    lines = ["@<TRIPOS>MOLECULE\n", "@<TRIPOS>ATOM\n"] # list to store all lines for a single pharmacophore
    line = ""
    for i, element in enumerate(pharmacophore.elements):
        if i <10:
            element_inx = " " + str(i)
        else:
            element_inx = str(i + 1)
        line += "\t" + element_inx + " "
        line += pharmagist_element_name[element.feature_name] + "\t"
        # Get point coordinates
        center = np.around(puw.get_value(element.center, to_unit="angstroms"), 4)
        # Add a space if positive
        if center[0] > 0: 
            x = " " + str(center[0]) 
        else: 
            x = str(center[0])
        if center[1] > 0: 
            y = " " + str(center[1]) 
        else: 
            y = str(center[1])
        if center[2] > 0: 
            z = " " + str(center[2]) 
        else:
            z = str(center[2])
        line += x + "\t" + y + "\t" + z + " "
        line += pharmagist_element_specs[element.feature_name] + "\t"
        line += str(i) + "\t"
        line += pharmagist_element_specs[element.feature_name] + "\t" + "0.0000\n"
        lines.append(line)
        line = ""

    lines.append("@<TRIPOS>BOND\n")
    for l in lines:
        doc.append(l)
    
with open(f_path, "w") as f:
    f.writelines(doc)


In [45]:
# Same function but justifying text

f_path = "../Data/pharmagist/Pharmacophore/saves/pharmacophore.mol2"

pharmagist_element_name = { # dictionary to map openphamracohpore feature names to pharmagist 
        "aromatic ring": "AR ",
        "hydrophobicity": "HYD",
        "hb acceptor": "ACC",
        "hb donor": "DON",
        "positive charge": "CAT",
        "negative charge": "ANI",
        }

pharmagist_element_specs = { 
        "aromatic ring": "AR ",
        "hydrophobicity": "HYD",
        "hb acceptor": "HB ",
        "hb donor": "HB ",
        "positive charge": "CHG",
        "negative charge": "CHG",
}

doc = [] # list to store all pharmacophores
for pharmacophore in pharmacophores:
    lines = ["@<TRIPOS>MOLECULE\n", "@<TRIPOS>ATOM\n"] # list to store all lines for a single pharmacophore
    line = ""
    for i, element in enumerate(pharmacophore.elements):
        element_inx = str(i + 1)
        line += element_inx.rjust(7)
        feat_name = pharmagist_element_name[element.feature_name]
        line += " " + feat_name
        # Get point coordinates
        center = np.around(puw.get_value(element.center, to_unit="angstroms"), 4)
        # Pad coordinates with zeros to the right. Number of zeros depends on sign
        if center[0] < 0:
            x = str(center[0]).ljust(7,"0").rjust(16)
        else:
            x = str(center[0]).ljust(6,"0").rjust(16)
        if center[1] < 0:
            y = str(center[1]).ljust(7,"0").rjust(10)
        else:
            y = str(center[1]).ljust(6,"0").rjust(10)
        if center[2] < 0:
            z = str(center[2]).ljust(7,"0").rjust(10)
        else:
            z = str(center[2]).ljust(6,"0").rjust(10)
        line += x + y + z + " "
        line += pharmagist_element_specs[element.feature_name].rjust(5)
        line += str(i).rjust(5)
        line += pharmagist_element_specs[element.feature_name].rjust(6)
        line += "0.0000\n".rjust(12)
        print(line)
        lines.append(line)
        line = ""

    lines.append("@<TRIPOS>BOND\n")
    for l in lines:
        doc.append(l)
    
with open(f_path, "w") as f:
    f.writelines(doc)


      1 DON          0.9119   -6.0590    0.6348   HB     0   HB      0.0000

      2 DON          1.0513   -6.2038    1.8705   HB     1   HB      0.0000

      3 DON          4.2877   -2.0367   -1.1797   HB     2   HB      0.0000

      4 ACC          0.7473   -5.1767    4.1427   HB     3   HB      0.0000

      5 ACC         -0.5019   -6.1897    5.7275   HB     4   HB      0.0000

      6 ACC          4.2877   -2.0367   -1.1797   HB     5   HB      0.0000

      7 ANI          0.0029   -6.1459    4.5449   CHG    6   CHG     0.0000

      8 AR          -0.5664   -8.3094    2.7476   AR     7   AR      0.0000

      9 AR           2.5683   -3.9947   -0.2701   AR     8   AR      0.0000

      1 DON          0.9119   -6.0590    0.6348   HB     0   HB      0.0000

      2 DON          1.0513   -6.2038    1.8705   HB     1   HB      0.0000

      3 DON          4.2877   -2.0367   -1.1797   HB     2   HB      0.0000

      4 ACC          0.7473   -5.1767    4.1427   HB     3   HB      0.0000
